In [18]:
import pandas as pd
pd.options.display.max_columns = None

import json
import pickle

In [2]:
tournament_id = 89132

In [3]:
with open(f"../data/raw/{tournament_id}.pkl", 'rb') as f:
    data = pickle.load(f)

In [4]:
data['tournament_info']

{'DateRange': 'Aug 9-10, 2025',
 'EndDate': '2025-08-10',
 'Country': 'United States',
 'Location': 'Perkasie, Pennsylvania',
 'LocationShort': 'Perkasie, PA',
 'Divisions': [{'DivisionID': 100,
   'Division': 'MPO',
   'DivisionName': 'Mixed Pro Open',
   'Players': 71,
   'IsPro': True,
   'ShortName': 'Mixed Pro Open',
   'AbbreviatedName': 'Mixed Pro Open',
   'LatestRound': 2},
  {'DivisionID': 105,
   'Division': 'FPO',
   'DivisionName': "Women's Pro Open",
   'Players': 14,
   'IsPro': True,
   'ShortName': "Women's Pro Open",
   'AbbreviatedName': "Women's Pro Open",
   'LatestRound': 2},
  {'DivisionID': 111,
   'Division': 'MP40',
   'DivisionName': 'Mixed Pro 40+',
   'Players': 21,
   'IsPro': True,
   'ShortName': 'Mixed Pro 40+',
   'AbbreviatedName': 'Mixed Pro 40+',
   'LatestRound': 2},
  {'DivisionID': 400,
   'Division': 'MA1',
   'DivisionName': 'Mixed Amateur 1',
   'Players': 38,
   'IsPro': False,
   'ShortName': 'Mixed Amateur 1',
   'AbbreviatedName': 'Mixed A

In [5]:
data['results'][0]

{'TournID': 89132,
 'ResultID': 211586222,
 'FirstName': 'Harry',
 'LastName': 'Chace',
 'PDGANum': 131546,
 'Division': 'MPO',
 'Pool': None,
 'Place': 1,
 'PlaceRank': 2,
 'ToPar': -11,
 'Tied': False,
 'Prize': '$660',
 'DNF': False,
 'Total': 120,
 'Name': 'Harry Chace',
 'Avatar': None,
 'ProfileURL': 'https://www.pdga.com/player/131546',
 'DivisionName': 'Mixed Pro Open',
 'FormattedPlace': '1st',
 'TierLetter': 'B',
 'TierType': 'Tier',
 'Class': 'Professional',
 'Rating': 1023,
 'RatingEffectiveDate': '2025-08-12',
 'Location': 'East Greenwich, RI',
 'Country': 'US',
 'Nationality': 'US',
 'AvatarURL': None,
 'Scores': [{'ScoreID': 25450418,
   'ThrowStatusID': None,
   'Round': 1,
   'RoundDate': '2025-08-10',
   'RoundField': 'Rd1',
   'TeeTime': '14:36:00',
   'TeeStart': '1',
   'RoundScore': 61,
   'ScoreToPar': -7,
   'RoundToPar': -7,
   'Birdies': 8,
   'BirdieHoles': 8,
   'Bogeys': 1,
   'BogeyHoles': 1,
   'HotRound': False,
   'RoundComplete': True,
   'RoundRating'

In [6]:
def parse_score(score):
    hole_scores = score["HoleScores"]
    layout = score["Layout"]

    parsed_score = score.copy()
    del parsed_score["HoleScores"]
    del parsed_score["Layout"]

    # Extract layout features of interest
    parsed_score["CourseName"] = layout["CourseName"]
    parsed_score["LayoutName"] = layout["Name"]
    parsed_score["Holes"] = layout["Holes"]
    parsed_score["Par"] = layout["Par"]
    parsed_score["Length"] = layout["Length"]
    parsed_score["LengthUnits"] = layout["Units"]

    # Recast Hole scores
    for i in range(1, 19):
        parsed_score["Hole " + str(i)] = hole_scores[str(i)]
    
    return parsed_score


rounds = []

for result in data["results"]:

    player_info = result.copy()

    scores = result["Scores"]
    del player_info["Scores"]

    for score in result['Scores']:
        parsed_score = parse_score(score)
        rounds.append(player_info | parsed_score)

In [7]:
df = pd.DataFrame(rounds)

In [8]:
df[df["Division"] == "FPO"].head()

,TournID,ResultID,FirstName,LastName,PDGANum,Division,Pool,Place,PlaceRank,ToPar,Tied,Prize,DNF,Total,Name,Avatar,ProfileURL,DivisionName,FormattedPlace,TierLetter,TierType,Class,Rating,RatingEffectiveDate,Location,Country,Nationality,AvatarURL,AverageRoundRating,RatingDiff,ToParString,Teammates,ScoreID,ThrowStatusID,Round,RoundDate,RoundField,TeeTime,TeeStart,RoundScore,ScoreToPar,RoundToPar,Birdies,BirdieHoles,Bogeys,BogeyHoles,HotRound,RoundComplete,RoundRating,CourseName,LayoutName,Holes,Par,Length,LengthUnits,Hole 1,Hole 2,Hole 3,Hole 4,Hole 5,Hole 6,Hole 7,Hole 8,Hole 9,Hole 10,Hole 11,Hole 12,Hole 13,Hole 14,Hole 15,Hole 16,Hole 17,Hole 18
141,89132,212013544,Lori,Merriman,35616,FPO,None,5,2,15,False,$90,False,150,Lori Merriman,/files/styles/large/public/pictures/picture-14...,https://www.pdga.com/player/35616,Women's Pro Open,5th,B,Tier,Professional,906.0,2025-08-12,"Sewickley, PA",US,None,/files/styles/large/public/pictures/picture-14...,904,-2,+15,[],25431649,NaN,1,2025-08-10,Rd1,10:24:00,1,74,+6,6,1,1,7,6,False,True,946.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,6,4,2,4,3,4,3,5,6,4,4,3,4,6,4,4,4,4
142,89132,212013544,Lori,Merriman,35616,FPO,None,5,2,15,False,$90,False,150,Lori Merriman,/files/styles/large/public/pictures/picture-14...,https://www.pdga.com/player/35616,Women's Pro Open,5th,B,Tier,Professional,906.0,2025-08-12,"Sewickley, PA",US,None,/files/styles/large/public/pictures/picture-14...,904,-2,+15,[],25469460,822176.0,2,2025-08-10,Rd2,09:45:00,1,76,+9,9,2,2,11,6,False,True,861.0,The Preserve Disc Golf Course,LVO 2025 FPO,18,67,8188,Feet,3,3,3,4,4,4,3,8,4,5,5,5,6,5,3,4,3,4
143,89132,211777194,Eden,Fornoff,162735,FPO,None,2,2,10,False,$220,False,145,Eden Fornoff,/files/styles/large/public/pictures/picture-17...,https://www.pdga.com/player/162735,Women's Pro Open,2nd,B,Tier,Professional,929.0,2025-08-12,"Delta, PA",US,None,/files/styles/large/public/pictures/picture-17...,925,-4,+10,[],25431839,813923.0,1,2025-08-10,Rd1,10:48:00,1,75,+7,7,0,0,7,5,False,True,940.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,5,4,3,4,3,7,3,4,5,4,5,3,5,5,3,3,5,4
144,89132,211777194,Eden,Fornoff,162735,FPO,None,2,2,10,False,$220,False,145,Eden Fornoff,/files/styles/large/public/pictures/picture-17...,https://www.pdga.com/player/162735,Women's Pro Open,2nd,B,Tier,Professional,929.0,2025-08-12,"Delta, PA",US,None,/files/styles/large/public/pictures/picture-17...,925,-4,+10,[],25469458,822174.0,2,2025-08-10,Rd2,09:45:00,1,70,+3,3,5,5,8,6,False,True,910.0,The Preserve Disc Golf Course,LVO 2025 FPO,18,67,8188,Feet,5,3,3,2,3,5,3,4,4,4,2,5,5,3,5,4,4,6
145,89132,212009503,Leah,Tsinajinnie,139109,FPO,None,1,2,8,False,$320,False,143,Leah Tsinajinnie,/files/styles/large/public/pictures/picture-14...,https://www.pdga.com/player/139109,Women's Pro Open,1st,B,Tier,Professional,910.0,2025-08-12,"Atlanta, GA",US,None,/files/styles/large/public/pictures/picture-14...,934,24,+8,[],25431840,814266.0,1,2025-08-10,Rd1,10:48:00,1,76,+8,8,0,0,8,6,False,True,933.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,5,4,3,4,4,4,3,4,5,5,4,3,5,7,3,3,4,6


In [9]:
df["PlaceRank"].unique()

array([2, 1])

In [10]:
df[df["PlaceRank"] == 1]

,TournID,ResultID,FirstName,LastName,PDGANum,Division,Pool,Place,PlaceRank,ToPar,Tied,Prize,DNF,Total,Name,Avatar,ProfileURL,DivisionName,FormattedPlace,TierLetter,TierType,Class,Rating,RatingEffectiveDate,Location,Country,Nationality,AvatarURL,AverageRoundRating,RatingDiff,ToParString,Teammates,ScoreID,ThrowStatusID,Round,RoundDate,RoundField,TeeTime,TeeStart,RoundScore,ScoreToPar,RoundToPar,Birdies,BirdieHoles,Bogeys,BogeyHoles,HotRound,RoundComplete,RoundRating,CourseName,LayoutName,Holes,Par,Length,LengthUnits,Hole 1,Hole 2,Hole 3,Hole 4,Hole 5,Hole 6,Hole 7,Hole 8,Hole 9,Hole 10,Hole 11,Hole 12,Hole 13,Hole 14,Hole 15,Hole 16,Hole 17,Hole 18
32,89132,212018592,Ross,Hart,139475,MPO,None,70,1,936,True,,True,999,Ross Hart,/files/styles/large/public/pictures/picture-14...,https://www.pdga.com/player/139475,Mixed Pro Open,T-70th,B,Tier,Professional,944.0,2025-08-12,"Millburn, NJ",US,None,/files/styles/large/public/pictures/picture-14...,986,42,DNF,[],25445289,NaN,1,2025-08-10,Rd1,13:24:00,1,68,E,0,7,7,7,6,False,True,986.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,7,3,2,3,3,3,2,5,5,3,4,4,5,5,3,2,4,5
33,89132,212018592,Ross,Hart,139475,MPO,None,70,1,936,True,,True,999,Ross Hart,/files/styles/large/public/pictures/picture-14...,https://www.pdga.com/player/139475,Mixed Pro Open,T-70th,B,Tier,Professional,944.0,2025-08-12,"Millburn, NJ",US,None,/files/styles/large/public/pictures/picture-14...,986,42,DNF,[],25480017,NaN,2,2025-08-10,Rd2,13:48:00,1,999,+936,936,0,0,72,18,False,True,NaN,The Preserve Disc Golf Course,LVO 2025,18,63,9073,Feet,7,7,7,7,8,8,7,8,7,8,7,7,8,8,8,8,7,8
126,89132,211898980,Joshua,Bizzle,177435,MPO,None,70,1,8,True,,True,999,Joshua Bizzle,/files/styles/large/public/pictures/picture-19...,https://www.pdga.com/player/177435,Mixed Pro Open,T-70th,B,Tier,Professional,993.0,2025-08-12,"Landisburg, PA",US,None,/files/styles/large/public/pictures/picture-19...,933,-60,DNF,[],25438533,NaN,1,2025-08-10,Rd1,12:24:00,1,76,+8,8,2,2,10,8,False,True,933.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,6,4,4,4,3,4,3,5,5,5,3,3,4,8,2,4,4,5
179,89132,211916926,Steve,Braud,16027,MP40,None,20,1,937,True,,True,999,Steve Braud,None,https://www.pdga.com/player/16027,Mixed Pro 40+,T-20th,B,Tier,Professional,959.0,2025-07-08,"Quakertown, PA",US,None,None,979,20,DNF,[],25430932,NaN,1,2025-08-10,Rd1,10:00:00,1,69,+1,1,1,1,2,2,False,True,979.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,5,5,3,4,3,4,3,4,4,4,4,3,4,5,4,3,4,3
180,89132,211916926,Steve,Braud,16027,MP40,None,20,1,937,True,,True,999,Steve Braud,None,https://www.pdga.com/player/16027,Mixed Pro 40+,T-20th,B,Tier,Professional,959.0,2025-07-08,"Quakertown, PA",US,None,None,979,20,DNF,[],25473829,NaN,2,2025-08-10,Rd2,10:48:00,1,999,+936,936,19,5,5,4,False,True,NaN,The Preserve Disc Golf Course,LVO 2025,18,63,9073,Feet,4,3,3,3,4,4,3,4,3,5,5,4,4,0,0,0,0,0
205,89132,211586656,Greg,Cielosczyk,21111,MP40,None,20,1,12,True,,True,999,Greg Cielosczyk,None,https://www.pdga.com/player/21111,Mixed Pro 40+,T-20th,B,Tier,Professional,938.0,2018-10-09,"Voorhees, NJ",US,None,None,907,-31,DNF,[],25427987,NaN,1,2025-08-10,Rd1,09:36:00,1,80,+12,12,2,2,14,8,False,True,907.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,7,4,3,3,3,4,3,7,7,5,4,3,6,6,2,4,5,4
260,89132,211586117,Ron Jon,Erb,237793,MA1,None,38,1,945,False,,True,999,Ron Jon Erb,/files/styles/large/public/pictures/picture-14...,https://www.pdga.com/player/237793,Mixed Amateur 1,38th,B,Tier,Professional,919.0,2025-08-12,"Pipersville, PA",US,None,/files/styles/large/public/pictures/picture-14...,900,-19,DNF,[],25442508,NaN,1,2025-08-10,Rd1,13:20:00,1,77,+14,14,0,0,14,9,False,True,900.0,The Preserve Disc Golf Course,LVO 2025,18,63,9073,Feet,4,3,3,4,4,5,5,4,3,4,3,5,8,4,5,5,3,5
261,89132,211586117,Ron Jon,Erb,237793,MA1,None,38,1,945,False,,True,999,Ron Jon Erb,/files/styles/large/public/pictures/picture-14...,https://www.pdga.com/player/237793,Mixed Amateur 1,38th,B,Tier,Professional,919.0,2025-08-12,"Pipersville, PA",US,None,/files/styles

## Drop some content-less columns

In [11]:
df.drop(["TournID", # all data from same tournament
        "Pool", # not sure what this even is
         "PlaceRank", # unsure what this should mean but looks to be duplicate of DNF
         "Name", # duplicated
        "TierLetter", "TierType", # describes what tier tournament is
         "Teammates", # completely empty; for doubles tournaments?
         "ThrowStatusID", # something for real-time scoring
         "RoundField", # duplicated information
         "HotRound", # all values of False - not properly encoded?
         "RoundComplete" # scope of this study is post-round analysis
        ], axis=1, inplace=True)

df.head()

,ResultID,FirstName,LastName,PDGANum,Division,Place,ToPar,Tied,Prize,DNF,Total,Avatar,ProfileURL,DivisionName,FormattedPlace,Class,Rating,RatingEffectiveDate,Location,Country,Nationality,AvatarURL,AverageRoundRating,RatingDiff,ToParString,ScoreID,Round,RoundDate,TeeTime,TeeStart,RoundScore,ScoreToPar,RoundToPar,Birdies,BirdieHoles,Bogeys,BogeyHoles,RoundRating,CourseName,LayoutName,Holes,Par,Length,LengthUnits,Hole 1,Hole 2,Hole 3,Hole 4,Hole 5,Hole 6,Hole 7,Hole 8,Hole 9,Hole 10,Hole 11,Hole 12,Hole 13,Hole 14,Hole 15,Hole 16,Hole 17,Hole 18
0,211586222,Harry,Chace,131546,MPO,1,-11,False,$660,False,120,None,https://www.pdga.com/player/131546,Mixed Pro Open,1st,Professional,1023.0,2025-08-12,"East Greenwich, RI",US,US,None,1031,8,-11,25450418,1,2025-08-10,14:36:00,1,61,-7,-7,8,8,1,1,1032.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,4,4,3,4,2,3,3,4,4,3,3,2,5,5,2,3,3,4
1,211586222,Harry,Chace,131546,MPO,1,-11,False,$660,False,120,None,https://www.pdga.com/player/131546,Mixed Pro Open,1st,Professional,1023.0,2025-08-12,"East Greenwich, RI",US,US,None,1031,8,-11,25483066,2,2025-08-10,14:36:00,1,59,-4,-4,7,7,3,3,1030.0,The Preserve Disc Golf Course,LVO 2025,18,63,9073,Feet,3,3,3,2,3,4,4,4,4,4,3,2,3,4,3,3,2,5
2,211586679,Dylan,Horst,32599,MPO,5,-7,True,$230,False,124,None,https://www.pdga.com/player/32599,Mixed Pro Open,T-5th,Professional,1008.0,2025-08-12,"West Chester, PA",US,None,None,1017,9,-7,25449101,1,2025-08-10,14:00:00,1,62,-6,-6,7,7,1,1,1025.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,4,4,2,3,3,3,3,4,4,4,3,2,5,4,3,3,4,4
3,211586679,Dylan,Horst,32599,MPO,5,-7,True,$230,False,124,None,https://www.pdga.com/player/32599,Mixed Pro Open,T-5th,Professional,1008.0,2025-08-12,"West Chester, PA",US,None,None,1017,9,-7,25483067,2,2025-08-10,14:36:00,1,62,-1,-1,4,4,3,3,1008.0,The Preserve Disc Golf Course,LVO 2025,18,63,9073,Feet,3,4,2,2,3,4,4,4,3,4,3,3,5,4,4,3,3,4
4,211653303,Mike,Carman,48559,MPO,2,-9,True,$420,False,122,/files/styles/large/public/pictures/picture-18...,https://www.pdga.com/player/48559,Mixed Pro Open,T-2nd,Professional,992.0,2025-08-12,"Ottsville, PA",US,None,/files/styles/large/public/pictures/picture-18...,1024,32,-9,25449681,1,2025-08-10,14:24:00,1,62,-6,-6,7,7,1,1,1025.0,Nockamixon State Park,LVO 2025,18,68,9325,Feet,5,4,2,4,3,3,4,3,4,4,3,2,4,4,2,3,4,4


In [17]:
metadata = {}

metadata['tournament_info'] = data['tournament_info']
metadata['live_layout'] = data['live_layout']

## Save data

In [13]:
#with open(f"../data/interim/{tournament_id}-interim.pkl", 'wb') as f:
    #pickle.dump(df, f)

In [22]:
with open(f"../data/interim/{tournament_id}-metadata-interim.json", 'w') as f:
    json.dump(metadata, f)